## Import

In [ ]:
%pip install tensorflow

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization as bn
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import model_from_json


In [ ]:
import keras.backend as K
import tensorflow as tf

In [ ]:
!pip install nibabel

In [ ]:
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
cd /content/drive/My Drive/Dataset/Train/

In [ ]:
img_path = glob("volume-*.nii")
mask_path = glob("segmentation-*.nii")

print("Number of images :", len(img_path))

In [ ]:
img_path

In [ ]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
img_path.sort(key=natural_keys)
mask_path.sort(key=natural_keys)

In [ ]:
# img_path

## Utils

In [ ]:
def normalize_hu(volume, clip_min=-100, clip_max=400):
    volume = np.clip(volume, clip_min, clip_max)
    return (volume - clip_min) / (clip_max - clip_min)


In [ ]:
patch_ratio = []

for i in range(16 + 1):
  patch_ratio.append(32 * i)

In [ ]:
patch_ratio

In [ ]:
def patch_sampling(img, mask, patch_ratio, pos_neg_ratio, threshold):

  temp_mask = mask

  temp_mask[temp_mask == 1] = 0
  temp_mask[temp_mask == 2] = 1

  positive_patch = []
  positive_mask = []

  negative_patch = []
  negative_mask = []

  negative_set = []


  for i in range(temp_mask.shape[2]):
    for x_bin in range(2, len(patch_ratio)):
        for y_bin in range(2, len(patch_ratio)):
          img_patch = img[patch_ratio[x_bin-2] : patch_ratio[x_bin], patch_ratio[y_bin - 2] : patch_ratio[y_bin], i]
          mask_patch = temp_mask[patch_ratio[x_bin-2] : patch_ratio[x_bin], patch_ratio[y_bin - 2] : patch_ratio[y_bin], i]
          _, count = np.unique(mask_patch, return_counts = True)

          if len(count) == 2:
            mask_percentage = count[1] / sum(count) * 100

            if threshold < mask_percentage :
              positive_patch.append(img_patch)
              positive_mask.append(mask_patch)


          elif len(count) ==1:

            temp_list = []
            temp_list.append(img_patch)
            temp_list.append(mask_patch)

            negative_set.append(temp_list)

  shuffle(negative_set)

  negative_set_to_use = negative_set[:len(positive_patch) * pos_neg_ratio]
  for negative_set in negative_set_to_use:
    negative_patch.append(negative_set[0])
    negative_mask.append(negative_set[1])

  negative_set_to_use = []

  return positive_patch, positive_mask, negative_patch, negative_mask


In [ ]:
def slice_to_patch(slice, patch_ratio):

  slice[slice == 1] = 0
  slice[slice == 2] = 1

  patch_list = []

  for x_bin in range(2, len(patch_ratio)):
    for y_bin in range(2, len(patch_ratio)):
      patch = slice[patch_ratio[x_bin-2] : patch_ratio[x_bin], patch_ratio[y_bin - 2] : patch_ratio[y_bin]]
      patch = patch.reshape(patch.shape + (1,))
      patch_list.append(patch)

  return np.array(patch_list)

In [ ]:
def patch_to_slice(patch, patch_ratio, input_shape, conf_threshold):

  slice = np.zeros((512, 512, 1))
  row_idx = 0
  col_idx = 0

  for i in range(len(patch)):

    slice[patch_ratio[row_idx]:patch_ratio[row_idx + 2], patch_ratio[col_idx]:patch_ratio[col_idx + 2]][patch[i] > conf_threshold] = 1

    col_idx += 1

    if i != 0 and (i+1) % 15 == 0:
      row_idx += 1
      col_idx = 0

  return slice

In [ ]:



from keras.saving import register_keras_serializable

@register_keras_serializable()
def weighted_binary_crossentropy(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    pos_weight = 0.90
    neg_weight = 0.10
    loss = - (y_true * K.log(y_pred) * pos_weight + (1 - y_true) * K.log(1 - y_pred) * neg_weight)
    return K.mean(loss)



In [ ]:
# Reference : https://github.com/dk67604/LITS-Challenge-Liver-Segmentation/blob/master/experiments/keras_realtime_train.ipynb

smooth = 1.
@register_keras_serializable()
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


## Model

In [ ]:
# input_shape = [64, 64, 1]
# dropout_rate = 0.3
# l2_lambda = 0.0002

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout, Dense, LayerNormalization, Reshape, MultiHeadAttention, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import register_keras_serializable


@register_keras_serializable()
class PatchExtractor(Layer):
    def __init__(self, patch_size,**kwargs):
        super(PatchExtractor, self).__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        patches = tf.image.extract_patches(
            images=inputs,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({
            "patch_size": self.patch_size
        })
        return config


class TransformerBlock(Layer):
    def __init__(self, num_heads, projection_dim, dropout_rate=0.1,**kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.projection_dim = projection_dim
        self.dropout_rate = dropout_rate

        self.mha = MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim // num_heads, dropout=0.0
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dense1 = Dense(projection_dim * 2, activation=tf.nn.gelu)
        self.dense2 = Dense(projection_dim)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = self.layernorm1(inputs)
        attention_output = self.mha(x, x)
        x1 = attention_output + inputs

        x2 = self.layernorm2(x1)
        x3 = self.dense1(x2)
        x3 = self.dropout1(x3, training=training)
        x3 = self.dense2(x3)
        x3 = self.dropout2(x3, training=training)

        return x1 + x3

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "projection_dim": self.projection_dim,
            "dropout_rate": self.dropout_rate
        })
        return config


class PositionalEmbedding(Layer):
    def __init__(self, sequence_length, projection_dim,**kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.sequence_length = sequence_length
        self.projection_dim = projection_dim
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=sequence_length, output_dim=projection_dim
        )

    def call(self, inputs):
        positions = tf.range(start=0, limit=self.sequence_length, delta=1)
        embedded_positions = self.position_embedding(positions)
        return inputs + embedded_positions

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "projection_dim": self.projection_dim
        })
        return config


class ReshapeToSpatial(Layer):
    def __init__(self, height, width, patch_size, channels,**kwargs):
        super(ReshapeToSpatial, self).__init__(**kwargs)
        self.height = height
        self.width = width
        self.patch_size = patch_size
        self.channels = channels

    def call(self, inputs):
        # Calculate dimensions
        h_dim = self.height // self.patch_size
        w_dim = self.width // self.patch_size

        # Reshape to spatial dimensions
        x = tf.reshape(inputs, [-1, h_dim, w_dim, self.patch_size * self.patch_size * self.channels])
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            "height": self.height,
            "width": self.width,
            "patch_size": self.patch_size,
            "channels": self.channels
        })
        return config


class PatchProjection(Layer):
    def __init__(self, projection_dim, l2_lambda=0.0002,**kwargs):
        super(PatchProjection, self).__init__(**kwargs)
        self.projection_dim = projection_dim
        self.l2_lambda = l2_lambda
        self.projection = Dense(
            units=projection_dim,
            kernel_regularizer=regularizers.l2(l2_lambda)
        )

    def call(self, inputs):
        return self.projection(inputs)

    def get_config(self):
        config = super().get_config()
        config.update({
            "projection_dim": self.projection_dim,
            "l2_lambda": self.l2_lambda
        })
        return config


def TransUNet(input_shape=(64, 64, 1), dropout_rate=0.3, l2_lambda=0.0002,
              projection_dim=64, transformer_layers=4, num_heads=4):
    inputs = Input(shape=input_shape, name="input")

    # === Encoder (same depth as U-Net) ===
    conv1 = Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D()(conv1)

    conv2 = Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D()(conv2)

    conv3 = Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D()(conv3)

    conv4 = Conv2D(256, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D()(conv4)

    conv5 = Conv2D(512, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(conv5)
    conv5 = BatchNormalization()(conv5)
    pool5 = MaxPooling2D()(conv5)  # (2, 2, 512)

    # === Bottleneck with Vision Transformer ===
    patch_size = 1  # Since pool5 is already 2x2, use 1x1 patches
    num_patches = (2 // patch_size) ** 2  # 4 patches
    feature_dim = pool5.shape[-1]

    # Extract patches
    patches = PatchExtractor(patch_size=patch_size)(pool5)

    # Project patches to the transformer dimension
    projected_patches = PatchProjection(projection_dim=projection_dim, l2_lambda=l2_lambda)(patches)

    # Add positional embeddings
    x = PositionalEmbedding(sequence_length=num_patches, projection_dim=projection_dim)(projected_patches)

    # Apply transformer blocks
    for i in range(transformer_layers):
        x = TransformerBlock(
            num_heads=num_heads,
            projection_dim=projection_dim,
            dropout_rate=dropout_rate
        )(x, training=True)

    # Reshape back to spatial dimensions for the decoder
    x = Reshape((2, 2, projection_dim))(x)

    # === Decoder ===
    up6 = Conv2DTranspose(512, 3, strides=2, padding='same')(x)  # 4x4
    up6 = concatenate([up6, conv5])
    up6 = Conv2D(512, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up6)
    up6 = BatchNormalization()(up6)
    up6 = Conv2D(512, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up6)
    up6 = BatchNormalization()(up6)

    up7 = Conv2DTranspose(256, 3, strides=2, padding='same')(up6)  # 8x8
    up7 = concatenate([up7, conv4])
    up7 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up7)
    up7 = BatchNormalization()(up7)
    up7 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up7)
    up7 = BatchNormalization()(up7)

    up8 = Conv2DTranspose(128, 3, strides=2, padding='same')(up7)  # 16x16
    up8 = concatenate([up8, conv3])
    up8 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up8)
    up8 = BatchNormalization()(up8)
    up8 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up8)
    up8 = BatchNormalization()(up8)

    up9 = Conv2DTranspose(64, 3, strides=2, padding='same')(up8)  # 32x32
    up9 = concatenate([up9, conv2])
    up9 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up9)
    up9 = BatchNormalization()(up9)
    up9 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up9)
    up9 = BatchNormalization()(up9)

    up10 = Conv2DTranspose(32, 3, strides=2, padding='same')(up9)  # 64x64
    up10 = concatenate([up10, conv1])
    up10 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up10)
    up10 = BatchNormalization()(up10)
    up10 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda))(up10)
    up10 = BatchNormalization()(up10)

    outputs = Conv2D(1, 1, activation='sigmoid')(up10)

    return Model(inputs, outputs, name="TransUNet")


In [ ]:
input_shape = [64, 64, 1]
dropout_rate = 0.3
l2_lambda = 0.0002

In [ ]:
model = TransUNet(input_shape=input_shape, dropout_rate=dropout_rate, l2_lambda=l2_lambda)


In [ ]:
model.summary()

In [ ]:
!pip install pydot pydotplus graphviz


In [ ]:


tf.keras.utils.plot_model(
    model,
    to_file='model7.png',
    show_shapes=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96
)




## Training

__Data__

In [ ]:
import os
import numpy as np
import nibabel as nib

save_dir = "/content/patches/"  # Absolute path in Colab
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

for i in range(len(img_path) - 80):
    img_3D = nib.load(img_path[i]).get_fdata()
    mask_3D = nib.load(mask_path[i]).get_fdata()

    pos_patch, pos_mask, neg_patch, neg_mask = patch_sampling(img_3D, mask_3D, patch_ratio, 3, 3.0)

    # Save patches to disk
    np.save(os.path.join(save_dir, f"img_patches_{i}.npy"), np.array(pos_patch + neg_patch))
    np.save(os.path.join(save_dir, f"mask_patches_{i}.npy"), np.array(pos_mask + neg_mask))

    print(f"Step [{i+1}/{len(img_path)}] : Patches saved to disk")

print("Processing complete! All patches are saved.")


In [ ]:
import os
import numpy as np
import nibabel as nib

patch_ratio = [32 * i for i in range(17)]
save_dir = "/content/patches_tumor_only"
os.makedirs(save_dir, exist_ok=True)

img_path = sorted(glob("volume-*.nii"), key=natural_keys)
mask_path = sorted(glob("segmentation-*.nii"), key=natural_keys)

for i in range(len(img_path)-30):
    img_3D = normalize_hu(nib.load(img_path[i]).get_fdata())
    mask_3D = nib.load(mask_path[i]).get_fdata()

    pos_patch, pos_mask, neg_patch, neg_mask = patch_sampling_tumor_inside_liver(
        img_3D, mask_3D, patch_ratio, pos_neg_ratio=3, threshold=3.0
    )

    np.save(os.path.join(save_dir, f"img_patches_{i}.npy"), np.array(pos_patch + neg_patch))
    np.save(os.path.join(save_dir, f"mask_patches_{i}.npy"), np.array(pos_mask + neg_mask))

    print(f"✓ Saved patches for volume {i}")


In [ ]:
# import numpy as np
# import os

# save_dir = "/content/patches_tumor_only"  # or "/content/patches_tumor_only" if following Option 1

# # Gather all valid patch files
# img_files = sorted([f for f in os.listdir(save_dir) if f.startswith("img_patches_")])
# mask_files = sorted([f for f in os.listdir(save_dir) if f.startswith("mask_patches_")])

# X, Y = [], []

# for img_f, mask_f in zip(img_files, mask_files):
#     x = np.load(os.path.join(save_dir, img_f))
#     y = np.load(os.path.join(save_dir, mask_f))

#     # Sanity check
#     if x.shape != y.shape or x.shape[1:] != (64, 64):
#         print(f"Skipping mismatched: {img_f}, {mask_f}, shape: {x.shape}, {y.shape}")
#         continue

#     X.append(x)
#     Y.append(y)

# # Final arrays
# X = np.concatenate(X, axis=0).astype(np.float32).reshape(-1, 64, 64, 1)
# Y = np.concatenate(Y, axis=0).astype(np.float32).reshape(-1, 64, 64, 1)

# print(f"✓ Loaded {X.shape[0]} patches.")
# print("X shape:", X.shape)
# print("Y shape:", Y.shape)

# # Optional: check how many positive pixels exist (tumor presence)
# tumor_ratio = Y.sum() / np.prod(Y.shape)
# print(f"Tumor pixel ratio: {tumor_ratio:.4f} (should not be ~0)")


In [ ]:
# # Save as .npy for quick reloading
# np.save("/content/final_tumor_patches1.npy", X)
# np.save("/content/final_tumor_masks1.npy", Y)

# print("✅ Final patch and mask arrays saved.")


In [ ]:
# X = np.load("/content/final_tumor_patches1.npy")
# Y = np.load("/content/final_tumor_masks1.npy")

# print("✅ Loaded saved training data:", X.shape, Y.shape)


In [ ]:
import numpy as np
import os

save_dir = "/content/patches/"

# Get all saved patch files
patch_files = sorted([f for f in os.listdir(save_dir) if f.startswith("img_patches_")])
mask_files = sorted([f for f in os.listdir(save_dir) if f.startswith("mask_patches_")])

# Load patches and filter out any problematic files
patch_list = []
mask_list = []

for f in patch_files:
    data = np.load(os.path.join(save_dir, f))
    if len(data.shape) == 3:  # Ensure it has (num_patches, H, W)
        patch_list.append(data)
    else:
        print(f"Skipping {f}, unexpected shape: {data.shape}")

for f in mask_files:
    data = np.load(os.path.join(save_dir, f))
    if len(data.shape) == 3:
        mask_list.append(data)
    else:
        print(f"Skipping {f}, unexpected shape: {data.shape}")

# Ensure all patches have the same shape before concatenation
if len(patch_list) > 0 and len(mask_list) > 0:
    total_patch = np.concatenate(patch_list, axis=0)
    total_mask = np.concatenate(mask_list, axis=0)

    # Reshape to (num_patches, 64, 64, 1) if needed
    total_patch = total_patch.reshape((total_patch.shape[0], 64, 64, 1))
    total_mask = total_mask.reshape((total_mask.shape[0], 64, 64, 1))

    print(f"Total patches shape: {total_patch.shape}")
    print(f"Total masks shape: {total_mask.shape}")
else:
    print("No valid patches found!")


In [ ]:


total_patch = np.array(total_patch).reshape((len(total_patch), 64, 64, 1))
total_mask = np.array(total_mask).reshape((len(total_mask), 64, 64, 1))



In [ ]:
mask_3D.shape

In [ ]:
np.save("total_patch21.npy", total_patch)
np.save("total_mask21.npy", total_mask)

In [ ]:
cd /content/drive/My Drive/Dataset/

In [ ]:
import numpy as np;

In [ ]:
# total_patch=np.load("total_patch1.npy")
# total_mask=np.load("total_mask1.npy")
total_patch = np.load("total_patch21.npy").astype(np.float32)
total_mask = np.load("total_mask21.npy").astype(np.float32)


In [ ]:
total_mask.shape

__Training__

In [ ]:
# adam = Adam(learning_rate = 0.0001) # Changed 'lr' to 'learning_rate'

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.saving import register_keras_serializable

@register_keras_serializable()
def weighted_binary_crossentropy(y_true, y_pred):
    import tensorflow.keras.backend as K
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    pos_weight = 0.90
    neg_weight = 0.10
    loss = - (y_true * K.log(y_pred) * pos_weight + (1 - y_true) * K.log(1 - y_pred) * neg_weight)
    return K.mean(loss)

smooth = 1.
@register_keras_serializable()
def dice_coef(y_true, y_pred):
    import tensorflow.keras.backend as K
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Compile the model with the same loss and metrics
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss=weighted_binary_crossentropy, metrics=[dice_coef])

In [ ]:


import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

model.fit(total_patch, total_mask, batch_size=32, epochs=100)


#model = model.get_layer("model_7")

model_json = model.to_json()
with open("final21.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("updated21.weights.h5")  # ✅ Correct extension
print("Saved model to disk")

model.save("finalmodel21.keras")  # Saves architecture, weights, and optimizer state





In [ ]:
# model_json = model.to_json()
# with open("model_json.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model_weights.h5")
# print("Saved model to disk")

In [ ]:
# from tensorflow.keras.optimizers import Adam
# from keras.saving import register_keras_serializable
# import tensorflow.keras.backend as K


# @register_keras_serializable()
# def dice_coef(y_true, y_pred, smooth=1e-6):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [ ]:
# model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=[dice_coef])


In [ ]:
# from tensorflow.keras.utils import Sequence

# class PatchDataGenerator(Sequence):
#     def __init__(self, X, Y, batch_size=32, shuffle=True):
#         self.X = X
#         self.Y = Y
#         self.batch_size = batch_size
#         self.shuffle = shuffle
#         self.indices = np.arange(len(self.X))
#         if shuffle:
#             np.random.shuffle(self.indices)

#     def __len__(self):
#         return int(np.ceil(len(self.X) / self.batch_size))

#     def __getitem__(self, index):
#         batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
#         batch_X = self.X[batch_indices]
#         batch_Y = self.Y[batch_indices]
#         return batch_X, batch_Y

#     def on_epoch_end(self):
#         if self.shuffle:
#             np.random.shuffle(self.indices)


In [ ]:
# train_gen = PatchDataGenerator(X, Y, batch_size=32)

# model.fit(train_gen, epochs=20)


In [ ]:
# Save model architecture to JSON
model_json = model.to_json()
with open("model_json1.json", "w") as json_file:
    json_file.write(model_json)

# Save weights with the required ".weights.h5" extension
model.save_weights("model_weights1.weights.h5")  # ✅ Correct extension
print("Saved model weights to disk")


## Save


In [ ]:
model.save("model21.keras")  # Saves the model in TensorFlow's new format
print("Saved entire model to disk")


In [ ]:
# model_json = model.to_json()
# with open("model_json.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model_weights.h5")
# print("Saved model to disk")

In [ ]:
model.summary()

## Load

In [ ]:
cd /content/drive/My Drive/Dataset/

In [ ]:
# json_file = open('model_json1.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model_weights1.weights.h5")
# print("Loaded model from disk")


from tensorflow.keras.models import model_from_json

# Register your custom layers
custom_objects = {
    "PatchExtractor": PatchExtractor,
    "TransformerBlock": TransformerBlock,
    "PositionalEmbedding": PositionalEmbedding,
    "ReshapeToSpatial": ReshapeToSpatial,
    "PatchProjection": PatchProjection
}

# Load model architecture from JSON
with open('model_json21.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json, custom_objects=custom_objects)

# Load weights
loaded_model.load_weights("model_weights21.weights.h5")

print("✅ Loaded model with custom layers from disk")




In [ ]:
loaded_model.summary()

## Example

__Ex.1 : volum-25.nii __

In [ ]:
cd /content/drive/My Drive/Dataset/Train

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def dice_score_np(y_true, y_pred, smooth=1e-6):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

dice_scores = []  # List to store Dice scores

In [ ]:
img_ex = nib.load(img_path[10]).get_fdata()
mask_ex = nib.load(mask_path[10]).get_fdata()

In [ ]:
img_path='/content/drive/My Drive/Dataset/Train/volume-9.nii'
img_ex = nib.load(img_path).get_fdata()
mask_path='/content/drive/My Drive/Dataset/Train/segmentation-9.nii'
mask_ex = nib.load(mask_path).get_fdata()

In [ ]:
limit = 0.75
top_slices = []

mask_ex[mask_ex == 1] = 0

for i in range(mask_ex.shape[2]):
    _, count = np.unique(mask_ex[:, :, i], return_counts=True)

    if len(count) > 1 and count[1] > 300:
        patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
        prediction = loaded_model.predict(patch_ex)
        prediction_mask = patch_to_slice(prediction, patch_ratio, input_shape, conf_threshold=0.98)

        pred_bin = (prediction_mask > 0.5).astype(np.uint8)
        true_bin = (mask_ex[:, :, i] > 0).astype(np.uint8)

        dice = dice_score_np(true_bin, pred_bin)

        if dice >= limit:
            top_slices.append({
                'dice': dice,
                'index': i,
                'image': img_ex[:, :, i],
                'true_mask': true_bin,
                'pred_mask': pred_bin.reshape((512, 512))
            })

# Sort by Dice score in descending order
top_slices = sorted(top_slices, key=lambda x: x['dice'], reverse=True)

# Print highest Dice score if available
if top_slices:
    print(f"\n Highest Dice Score: {top_slices[0]['dice']:.4f}\n")
else:
    print("❌ No slices found")

# Display top 3 slices with Dice ≥ 0.79
for idx, item in enumerate(top_slices[:3]):
    print(f"Top {idx+1} | Slice {item['index']} | Dice Score: {item['dice']:.4f}")

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 15))
    ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    ax1.set_title("Image", fontsize="x-large")
    ax1.grid(False)

    ax2.imshow(np.rot90(item['true_mask'], 3), cmap='bone')
    ax2.set_title("Mask (True)", fontsize="x-large")
    ax2.grid(False)

    ax3.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    ax3.set_title(f"Mask (Pred)\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax3.grid(False)

    plt.show()


Only Tumor overlay

In [ ]:
img_ex = nib.load(img_path[10]).get_fdata()
mask_ex = nib.load(mask_path[10]).get_fdata()

In [ ]:
limit = 0.75
top_slices = []

# Make a copy of the mask to preserve the original
tumor_mask = mask_ex.copy()
tumor_mask[tumor_mask == 1] = 0  # Remove liver labels (keep only tumor)

for i in range(mask_ex.shape[2]):
    _, count = np.unique(tumor_mask[:, :, i], return_counts=True)

    if len(count) > 1 and count[1] > 300:
        patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
        prediction = loaded_model.predict(patch_ex)
        prediction_mask = patch_to_slice(prediction, patch_ratio, (512, 512), conf_threshold=0.98)

        pred_bin = (prediction_mask > 0.5).astype(np.uint8)
        true_bin = (tumor_mask[:, :, i] > 0).astype(np.uint8)

        dice = dice_score_np(true_bin, pred_bin)

        if dice >= limit:
            # Create HU-windowed version of the CT slice
            ct_slice = img_ex[:, :, i]
            liver_min, liver_max = -100, 200  # Liver window
            hu_windowed = np.clip(ct_slice, liver_min, liver_max)
            hu_windowed = (hu_windowed - liver_min) / (liver_max - liver_min)

            top_slices.append({
                'dice': dice,
                'index': i,
                'image': ct_slice,
                'hu_windowed': hu_windowed,
                'true_mask': true_bin,
                'pred_mask': pred_bin.reshape((512, 512))
            })

# Sort by Dice score in descending order
top_slices = sorted(top_slices, key=lambda x: x['dice'], reverse=True)

# Print highest Dice score if available
if top_slices:
    print(f"\n Highest Dice Score: {top_slices[0]['dice']:.4f}\n")
else:
    print("❌ No slices found")

# Display top 3 slices with Dice ≥ limit
for idx, item in enumerate(top_slices[:3]):
    print(f"Top {idx+1} | Slice {item['index']} | Dice Score: {item['dice']:.4f}")

    # Create a 1x4 grid for all visualizations
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))

    # Original CT image
    ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    ax1.set_title("Original CT", fontsize="x-large")
    ax1.grid(False)

    # HU-windowed CT
    ax2.imshow(np.rot90(item['hu_windowed'], 3), cmap='gray')
    ax2.set_title("HU-Windowed CT", fontsize="x-large")
    ax2.grid(False)

    # True tumor mask
    ax3.imshow(np.rot90(item['true_mask'], 3), cmap='bone')
    ax3.set_title("Mask (True)", fontsize="x-large")
    ax3.grid(False)

    # Predicted tumor mask
    ax4.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    ax4.set_title(f"Mask (Pred)\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax4.grid(False)

    plt.tight_layout()
    plt.show()

    # Add a second visualization showing the tumor overlay on the HU-windowed CT
    fig, ax = plt.subplots(figsize=(8, 8))

    # Create a colored version of the HU-windowed CT image
    overlay = np.stack([item['hu_windowed'], item['hu_windowed'], item['hu_windowed']], axis=2)

    # Highlight tumor prediction in red
    tumor_pixels = item['pred_mask'].astype(bool)
    overlay[tumor_pixels] = [1.0, 0.3, 0.3]  # Red for tumor

    # Display the overlay
    ax.imshow(np.rot90(overlay, 3))
    ax.set_title(f"Tumor Overlay on HU-Windowed CT\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax.grid(False)

    plt.tight_layout()
    plt.show()

In [ ]:
limit = 0.75
top_slices = []

# Make a copy of the mask to preserve the original
tumor_mask = mask_ex.copy()
tumor_mask[tumor_mask == 1] = 0  # Remove liver labels (keep only tumor)

for i in range(mask_ex.shape[2]):
    _, count = np.unique(tumor_mask[:, :, i], return_counts=True)

    if len(count) > 1 and count[1] > 300:
        patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
        prediction = loaded_model.predict(patch_ex)
        prediction_mask = patch_to_slice(prediction, patch_ratio, (512, 512), conf_threshold=0.98)

        pred_bin = (prediction_mask > 0.5).astype(np.uint8)
        true_bin = (tumor_mask[:, :, i] > 0).astype(np.uint8)

        dice = dice_score_np(true_bin, pred_bin)

        if dice >= limit:
            # Create HU-windowed version of the CT slice
            ct_slice = img_ex[:, :, i]
            liver_min, liver_max = -100, 200  # Liver window
            hu_windowed = np.clip(ct_slice, liver_min, liver_max)
            hu_windowed = (hu_windowed - liver_min) / (liver_max - liver_min)

            # Get the liver mask from the original mask (assuming 1 is liver)
            liver_mask = (mask_ex[:, :, i] == 1).astype(np.uint8)

            top_slices.append({
                'dice': dice,
                'index': i,
                'image': ct_slice,
                'hu_windowed': hu_windowed,
                'true_mask': true_bin,
                'liver_mask': liver_mask,
                'pred_mask': pred_bin.reshape((512, 512))
            })

# Sort by Dice score in descending order
top_slices = sorted(top_slices, key=lambda x: x['dice'], reverse=True)

# Print highest Dice score if available
if top_slices:
    print(f"\n Highest Dice Score: {top_slices[0]['dice']:.4f}\n")
else:
    print("❌ No slices found")

# Display top 3 slices with Dice ≥ limit
for idx, item in enumerate(top_slices[:3]):
    print(f"Top {idx+1} | Slice {item['index']} | Dice Score: {item['dice']:.4f}")

    # Create a 1x4 grid for all visualizations
    # fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))

    # # Original CT image
    # ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    # ax1.set_title("Original CT", fontsize="x-large")
    # ax1.grid(False)

    # # HU-windowed CT
    # ax2.imshow(np.rot90(item['hu_windowed'], 3), cmap='gray')
    # ax2.set_title("HU-Windowed CT", fontsize="x-large")
    # ax2.grid(False)

    # # True tumor mask
    # ax3.imshow(np.rot90(item['true_mask'], 3), cmap='bone')
    # ax3.set_title("Mask (True)", fontsize="x-large")
    # ax3.grid(False)

    # # Predicted tumor mask
    # ax4.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    # ax4.set_title(f"Mask (Pred)\nDice: {item['dice']:.4f}", fontsize="x-large")
    # ax4.grid(False)

    # plt.tight_layout()
    # plt.show()

    # # Add a visualization showing both liver and tumor overlay on the HU-windowed CT
    # fig, ax = plt.subplots(figsize=(16, 4))  # Match with above

    # # Create a colored version of the HU-windowed CT image
    # overlay = np.stack([item['hu_windowed'], item['hu_windowed'], item['hu_windowed']], axis=2)

    # # Highlight liver in blue (semi-transparent) - use valid color values (0-1)
    # liver_pixels = item['liver_mask'].astype(bool)
    # # Use multiplication factor that keeps values within 0-1 range
    # overlay[liver_pixels] = overlay[liver_pixels] * [0.7, 0.7, 1.0]  # Blue tint for liver (safe values)

    # # Highlight tumor prediction in red (more prominent)
    # tumor_pixels = item['pred_mask'].astype(bool)
    # overlay[tumor_pixels] = [1.0, 0.3, 0.3]  # Red for tumor

    # # Display the overlay with rotation
    # ax.imshow(np.rot90(overlay, 3))
    # ax.set_title(f"Liver & Tumor Overlay on HU-Windowed CT\nDice: {item['dice']:.4f}", fontsize="x-large")
    # ax.grid(False)

    # # Add a legend with valid color values
    # from matplotlib.patches import Patch
    # legend_elements = [
    #     Patch(facecolor=[0.7, 0.7, 1.0], label='Liver'),  # Fixed: 1.2 -> 1.0
    #     Patch(facecolor=[1.0, 0.3, 0.3], label='Tumor (Pred)')
    # ]
    # ax.legend(handles=legend_elements, loc='lower right')

    # plt.tight_layout()
    # plt.show()
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(16, 4))  # <- updated size

    # Original CT image
    ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    ax1.set_title("Original CT", fontsize="large")
    ax1.grid(False)

    # HU-windowed CT
    ax2.imshow(np.rot90(item['hu_windowed'], 3), cmap='gray')
    ax2.set_title("HU-Windowed CT", fontsize="large")
    ax2.grid(False)

    # True tumor mask
    ax3.imshow(np.rot90(item['true_mask'], 3), cmap='bone')
    ax3.set_title("Mask (True)", fontsize="large")
    ax3.grid(False)

    # Predicted tumor mask
    ax4.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    ax4.set_title(f"Mask (Pred)\nDice: {item['dice']:.4f}", fontsize="large")
    ax4.grid(False)

    plt.tight_layout()
    plt.show()

    # Create consistent overlay visualization (same width and height)
    fig, ax = plt.subplots(figsize=(16, 4))  # <- match size

    # Create color overlay
    overlay = np.stack([item['hu_windowed']] * 3, axis=2)

    # Apply liver (blue) overlay
    liver_pixels = item['liver_mask'].astype(bool)
    overlay[liver_pixels] = overlay[liver_pixels] * [0.7, 0.7, 1.0]

    # Apply tumor prediction (red)
    tumor_pixels = item['pred_mask'].astype(bool)
    overlay[tumor_pixels] = [1.0, 0.3, 0.3]

    # Show image
    ax.imshow(np.rot90(overlay, 3))
    ax.set_title(f"Liver & Tumor Overlay on HU-Windowed CT\nDice: {item['dice']:.4f}", fontsize="large")
    ax.grid(False)

    # Legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=[0.7, 0.7, 1.0], label='Liver'),
        Patch(facecolor=[1.0, 0.3, 0.3], label='Tumor (Pred)')
    ]
    ax.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    plt.show()



In [ ]:
img_ex = nib.load(img_path[10]).get_fdata()
mask_ex = nib.load(mask_path[10]).get_fdata()

In [ ]:
limit = 0.75
top_slices = []

# Make a copy of the mask to preserve the original
tumor_mask = mask_ex.copy()
tumor_mask[tumor_mask == 1] = 0  # Remove liver labels (keep only tumor)

for i in range(mask_ex.shape[2]):
    _, count = np.unique(tumor_mask[:, :, i], return_counts=True)

    if len(count) > 1 and count[1] > 300:
        patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
        prediction = loaded_model.predict(patch_ex)
        prediction_mask = patch_to_slice(prediction, patch_ratio, (512, 512), conf_threshold=0.98)

        pred_bin = (prediction_mask > 0.5).astype(np.uint8)
        true_bin = (tumor_mask[:, :, i] > 0).astype(np.uint8)

        dice = dice_score_np(true_bin, pred_bin)

        if dice >= limit:
            # Create HU-windowed version of the CT slice
            ct_slice = img_ex[:, :, i]
            liver_min, liver_max = -100, 200  # Liver window
            hu_windowed = np.clip(ct_slice, liver_min, liver_max)
            hu_windowed = (hu_windowed - liver_min) / (liver_max - liver_min)

            # Get the liver mask from the original mask (assuming 1 is liver)
            liver_mask = (mask_ex[:, :, i] == 1).astype(np.uint8)

            # Get tumor mask from original mask (assuming values > 1 are tumor)
            true_tumor_mask = true_bin

            top_slices.append({
                'dice': dice,
                'index': i,
                'image': ct_slice,
                'hu_windowed': hu_windowed,
                'liver_mask': liver_mask,
                'true_tumor_mask': true_tumor_mask,
                'pred_mask': pred_bin.reshape((512, 512))
            })

# Sort by Dice score in descending order
top_slices = sorted(top_slices, key=lambda x: x['dice'], reverse=True)

# Print highest Dice score if available
if top_slices:
    print(f"\n Highest Dice Score: {top_slices[0]['dice']:.4f}\n")
else:
    print("❌ No slices found")

# Display top 3 slices with Dice ≥ limit
for idx, item in enumerate(top_slices[:3]):
    print(f"Top {idx+1} | Slice {item['index']} | Dice Score: {item['dice']:.4f}")

    # Create a 1x3 grid for main visualizations
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    # Original CT image
    ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    ax1.set_title("Original CT", fontsize="x-large")
    ax1.grid(False)

    # HU-windowed CT
    ax2.imshow(np.rot90(item['hu_windowed'], 3), cmap='gray')
    ax2.set_title("HU-Windowed CT", fontsize="x-large")
    ax2.grid(False)

    # Predicted tumor mask
    ax3.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    ax3.set_title(f"Tumor Mask (Pred)\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax3.grid(False)

    plt.tight_layout()
    plt.show()

    # Add a visualization showing ground truth liver and tumor overlay on the HU-windowed CT
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    # 1. Ground Truth Overlay
    # Create a colored version of the HU-windowed CT image
    gt_overlay = np.stack([item['hu_windowed'], item['hu_windowed'], item['hu_windowed']], axis=2)

    # Highlight ground truth liver in blue
    liver_pixels = item['liver_mask'].astype(bool)
    gt_overlay[liver_pixels] = gt_overlay[liver_pixels] * [0.7, 0.7, 1.0]  # Blue tint for liver

    # Highlight ground truth tumor in red
    true_tumor_pixels = item['true_tumor_mask'].astype(bool)
    gt_overlay[true_tumor_pixels] = [1.0, 0.3, 0.3]  # Red for tumor

    # Display the ground truth overlay with rotation
    ax1.imshow(np.rot90(gt_overlay, 3))
    ax1.set_title("Ground Truth: Liver (Blue) & Tumor (Red)", fontsize="x-large")
    ax1.grid(False)

    # 2. Prediction Overlay
    # Create a colored version of the HU-windowed CT image
    pred_overlay = np.stack([item['hu_windowed'], item['hu_windowed'], item['hu_windowed']], axis=2)

    # Highlight ground truth liver in blue (same as in ground truth)
    pred_overlay[liver_pixels] = pred_overlay[liver_pixels] * [0.7, 0.7, 1.0]  # Blue tint for liver

    # Highlight predicted tumor in red
    pred_tumor_pixels = item['pred_mask'].astype(bool)
    pred_overlay[pred_tumor_pixels] = [1.0, 0.3, 0.3]  # Red for tumor

    # Display the prediction overlay with rotation
    ax2.imshow(np.rot90(pred_overlay, 3))
    ax2.set_title(f"Prediction: Liver (Blue) & Tumor (Red)\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax2.grid(False)

    # Add a legend with valid color values
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=[0.7, 0.7, 1.0], label='Liver'),
        Patch(facecolor=[1.0, 0.3, 0.3], label='Tumor')
    ]
    ax1.legend(handles=legend_elements, loc='lower right')
    ax2.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# --- Helper: HU Normalization ---
def normalize_hu(volume, clip_min=-100, clip_max=400):
    volume = np.clip(volume, clip_min, clip_max)
    return (volume - clip_min) / (clip_max - clip_min)

# --- Evaluate Model & Visualize Top Tumor Predictions ---
limit = 0.75
top_slices = []

# Remove liver label (keep only tumor)
tumor_mask = mask_ex.copy()
tumor_mask[tumor_mask == 1] = 0  # Remove liver
tumor_mask[tumor_mask == 2] = 1  # Tumor = 1

for i in range(mask_ex.shape[2]):
    _, count = np.unique(tumor_mask[:, :, i], return_counts=True)

    if len(count) > 1 and count[1] > 300:  # Tumor present
        patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
        prediction = loaded_model.predict(patch_ex)
        prediction_mask = patch_to_slice(prediction, patch_ratio, (512, 512), conf_threshold=0.98)

        pred_bin = (prediction_mask > 0.5).astype(np.uint8)
        true_bin = (tumor_mask[:, :, i] > 0).astype(np.uint8)

        dice = dice_score_np(true_bin, pred_bin)

        if dice >= limit:
            ct_slice = img_ex[:, :, i]  # Raw HU values
            hu_windowed = normalize_hu(ct_slice)  # ✅ HU-normalized

            liver_mask = (mask_ex[:, :, i] == 1).astype(np.uint8)
            true_tumor_mask = true_bin

            top_slices.append({
                'dice': dice,
                'index': i,
                'image': ct_slice,
                'hu_windowed': hu_windowed,
                'liver_mask': liver_mask,
                'true_tumor_mask': true_tumor_mask,
                'pred_mask': pred_bin.reshape((512, 512))
            })

# --- Sort by Dice ---
top_slices = sorted(top_slices, key=lambda x: x['dice'], reverse=True)

# --- Print Best Dice ---
if top_slices:
    print(f"\n Highest Dice Score: {top_slices[0]['dice']:.4f}\n")
else:
    print("❌ No slices found")

# --- Visualize Top 3 Slices ---
for idx, item in enumerate(top_slices[:3]):
    print(f"Top {idx+1} | Slice {item['index']} | Dice Score: {item['dice']:.4f}")

    # --- Main Views: CT, HU-windowed, Pred ---
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    ax1.imshow(np.rot90(item['image'], 3), cmap='bone')
    ax1.set_title("Original CT", fontsize="x-large")
    ax1.grid(False)

    ax2.imshow(np.rot90(item['hu_windowed'], 3), cmap='gray')
    ax2.set_title("HU-Normalized CT", fontsize="x-large")
    ax2.grid(False)

    ax3.imshow(np.rot90(item['pred_mask'], 3), cmap='bone')
    ax3.set_title(f"Tumor Prediction\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax3.grid(False)

    plt.tight_layout()
    plt.show()

    # --- Overlays: Ground Truth and Prediction ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

    hu_rgb = np.stack([item['hu_windowed']] * 3, axis=2)

    # --- Ground Truth Overlay ---
    gt_overlay = hu_rgb.copy()
    gt_overlay[item['liver_mask'].astype(bool)] *= [0.7, 0.7, 1.0]  # Blue for liver
    gt_overlay[item['true_tumor_mask'].astype(bool)] = [1.0, 0.3, 0.3]  # Red for tumor

    ax1.imshow(np.rot90(gt_overlay, 3))
    ax1.set_title("Ground Truth: Liver (Blue), Tumor (Red)", fontsize="x-large")
    ax1.axis('off')

    # --- Prediction Overlay ---
    pred_overlay = hu_rgb.copy()
    pred_overlay[item['liver_mask'].astype(bool)] *= [0.7, 0.7, 1.0]
    pred_overlay[item['pred_mask'].astype(bool)] = [1.0, 0.3, 0.3]

    ax2.imshow(np.rot90(pred_overlay, 3))
    ax2.set_title(f"Prediction: Liver (Blue), Tumor (Red)\nDice: {item['dice']:.4f}", fontsize="x-large")
    ax2.axis('off')

    # --- Legend ---
    legend_elements = [
        Patch(facecolor=[0.7, 0.7, 1.0], label='Liver'),
        Patch(facecolor=[1.0, 0.3, 0.3], label='Tumor')
    ]
    ax1.legend(handles=legend_elements, loc='lower right')
    ax2.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    plt.show()


In [ ]:
import nibabel as nib
from glob import glob
import os

# Assume files are already in a local directory (modify this for your environment)
# These should be NIfTI files from LiTS challenge
image_paths = sorted(glob("volume-*.nii"))  # Load only one for demo
mask_paths = sorted(glob("segmentation-*.nii"))

# Load the first volume and mask
img = nib.load(image_paths[10]).get_fdata()
mask = nib.load(mask_paths[10]).get_fdata()

# Extract HU values based on mask labels
liver_voxels = img[mask == 1]
tumor_voxels = img[mask == 2]

# Basic stats
liver_mean, liver_std = np.mean(liver_voxels), np.std(liver_voxels)
tumor_mean, tumor_std = np.mean(tumor_voxels), np.std(tumor_voxels)

# Plot real histogram
plt.figure(figsize=(10, 6))
plt.hist(liver_voxels, bins=60, alpha=0.6, label=f'Liver (mean={liver_mean:.1f})', color='blue', density=True)
plt.hist(tumor_voxels, bins=60, alpha=0.6, label=f'Tumor (mean={tumor_mean:.1f})', color='red', density=True)
plt.axvline(liver_mean, color='blue', linestyle='--')
plt.axvline(tumor_mean, color='red', linestyle='--')
plt.title("Real HU Distributions from LiTS Volume", fontsize=14)
plt.xlabel("Hounsfield Unit (HU)")
plt.ylabel("Normalized Frequency")
plt.legend()
plt.grid(True)
plt.show()


__Ex.2 : volum-129.nii __

In [ ]:
# img_ex = nib.load(img_path[54]).get_fdata()
# mask_ex = nib.load(mask_path[54]).get_fdata()

In [ ]:
# mask_ex[mask_ex == 1] = 0

# for i in range(mask_ex.shape[2]):
#     _, count = np.unique(mask_ex[:, :, i], return_counts=True)

#     if len(count) > 1 and count[1] > 300:

#         patch_ex = slice_to_patch(img_ex[:, :, i], patch_ratio)
#         prediction = loaded_model.predict(patch_ex)
#         prediction_mask = patch_to_slice(prediction, patch_ratio, input_shape, conf_threshold = 0.98)

#         fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize = ((15, 15)))

#         ax1.imshow(np.rot90(img_ex[:, :, i], 3), cmap = 'bone')
#         ax1.set_title("Image", fontsize = "x-large")
#         ax1.grid(False)
#         ax2.imshow(np.rot90(mask_ex[:, :, i], 3), cmap = 'bone')
#         ax2.set_title("Mask (True)", fontsize = "x-large")
#         ax2.grid(False)
#         ax3.imshow(np.rot90(prediction_mask.reshape((512, 512)), 3), cmap = 'bone')
#         ax3.set_title("Mask (Pred)", fontsize = "x-large")
#         ax3.grid(False)
#         plt.show()

__Ex.3 : volum-27.nii __